In [1]:
import os
import json
import importlib
import pandas as pd
from pydantic import ValidationError

# === параметры ===
INPUT_FILE = "data/cash_sample100_v3.xlsx"    # путь к Excel
PRODUCT = "cash"                           # jarir | cash | purchase
THRESHOLD = 0.3                             # порог классификации

# === автоматический импорт враппера ===
WRAPPER_MAPPER = {
    "jarir": "wrappers.jarir_wrapper",
    "cash": "wrappers.cash_wrapper",
    "purchase": "wrappers.purchase_wrapper",
}

wrapper_module = importlib.import_module(WRAPPER_MAPPER[PRODUCT])
ScoreModel = getattr(wrapper_module, "ScoreModel")
InputData = getattr(wrapper_module, "InputData")

# === загрузка данных ===
df = pd.read_excel(INPUT_FILE)
model = ScoreModel()

proba_list, decision_list, shap_json_list = [], [], []

for idx, row in df.iterrows():
    row_dict = row.to_dict()

    # подстраховка для occupationcode (актуально только для cash)
    if "occupationcode" in row_dict:
        row_dict["occupationcode"] = str(row_dict.get("occupationcode", ""))

    # === фикс для days_in_stage: если оно есть в данных, удаляем даты, чтобы не перезаписывать ===
    if "days_in_stage" in row_dict and pd.notna(row_dict["days_in_stage"]):
        row_dict.pop("date_of_joining", None)
        row_dict.pop("CreatedOn", None)

    try:
        input_data = InputData(**row_dict)
        _, result = model.predict(input_data, threshold=THRESHOLD)
        proba_list.append(result["probability_of_default"])
        decision_list.append(result["decision"])
        shap_json_list.append(json.dumps(result["shap_values"], ensure_ascii=False))
    except ValidationError as e:
        print(f"⚠️ Ошибка валидации в строке {idx}: {e}")
        proba_list.append(None)
        decision_list.append(None)
        shap_json_list.append(None)
    except Exception as e:
        print(f"⚠️ Ошибка в строке {idx}: {e}")
        proba_list.append(None)
        decision_list.append(None)
        shap_json_list.append(None)

# === добавляем результат ===
df["proba"] = proba_list
df["decision"] = decision_list
df["shap_values"] = shap_json_list

# === сохраняем результат ===
os.makedirs("predictions", exist_ok=True)
output_path = os.path.join("predictions", os.path.basename(INPUT_FILE).replace(".xlsx", "_predictions.xlsx"))
df.to_excel(output_path, index=False)

print(f"\n✅ Файл с предсказаниями сохранён в: {output_path}")


✅ Файл с предсказаниями сохранён в: predictions\cash_sample100_v3_predictions.xlsx
